In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import math
import random
from tqdm import tqdm_notebook
import matplotlib
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 30)

from datetime import datetime
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
path='../input/ashrae-energy-prediction/'
path_new = '../input/lag-transform/'
from sklearn.metrics import roc_auc_score,mean_squared_error
from sklearn.model_selection import StratifiedKFold,TimeSeriesSplit, train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from math import sin,log,pow
import lightgbm as lgb
import datetime
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder,StandardScaler, OneHotEncoder, MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import multiprocessing as mp
import multiprocessing as mp
from multiprocessing import Pool
from tqdm import tqdm
from functools import partial
pd.options.mode.chained_assignment = None
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
import holidays
from bayes_opt import BayesianOptimization
import time

import os
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))
def ls():
     print([v for v in globals().keys() if not v.startswith('_')])


../input/ashrae-energy-prediction/sample_submission.csv
../input/ashrae-energy-prediction/building_metadata.csv
../input/ashrae-energy-prediction/weather_test.csv
../input/ashrae-energy-prediction/train.csv
../input/ashrae-energy-prediction/test.csv
../input/ashrae-energy-prediction/weather_train.csv


In [2]:
def sd(col, max_loss_limit=0.001, avg_loss_limit=0.001, na_loss_limit=0, n_uniq_loss_limit=0, fillna=0):
    """
    max_loss_limit - don't allow any float to lose precision more than this value. Any values are ok for GBT algorithms as long as you don't unique values.
                     See https://en.wikipedia.org/wiki/Half-precision_floating-point_format#Precision_limitations_on_decimal_values_in_[0,_1]
    avg_loss_limit - same but calculates avg throughout the series.
    na_loss_limit - not really useful.
    n_uniq_loss_limit - very important parameter. If you have a float field with very high cardinality you can set this value to something like n_records * 0.01 in order to allow some field relaxing.
    """
    is_float = str(col.dtypes)[:5] == 'float'
    na_count = col.isna().sum()
    n_uniq = col.nunique(dropna=False)
    try_types = ['float16', 'float32']

    if na_count <= na_loss_limit:
        try_types = ['int8', 'int16', 'float16', 'int32', 'float32']

    for type in try_types:
        col_tmp = col

        # float to int conversion => try to round to minimize casting error
        if is_float and (str(type)[:3] == 'int'):
            col_tmp = col_tmp.copy().fillna(fillna).round()

        col_tmp = col_tmp.astype(type)
        max_loss = (col_tmp - col).abs().max()
        avg_loss = (col_tmp - col).abs().mean()
        na_loss = np.abs(na_count - col_tmp.isna().sum())
        n_uniq_loss = np.abs(n_uniq - col_tmp.nunique(dropna=False))

        if max_loss <= max_loss_limit and avg_loss <= avg_loss_limit and na_loss <= na_loss_limit and n_uniq_loss <= n_uniq_loss_limit:
            return col_tmp

    # field can't be converted
    return col


def reduce_mem_usage_sd(df, deep=True, verbose=False, obj_to_cat=False):
    numerics = ['int16', 'uint16', 'int32', 'uint32', 'int64', 'uint64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=deep).sum() / 1024 ** 2
    for col in tqdm_notebook(df.columns):
        col_type = df[col].dtypes

        # collect stats
        na_count = df[col].isna().sum()
        n_uniq = df[col].nunique(dropna=False)
        
        # numerics
        if col_type in numerics:
            df[col] = sd(df[col])

        # strings
        if (col_type == 'object') and obj_to_cat:
            df[col] = df[col].astype('category')
        
        if verbose:
            print(f'Column {col}: {col_type} -> {df[col].dtypes}, na_count={na_count}, n_uniq={n_uniq}')
        new_na_count = df[col].isna().sum()
        if (na_count != new_na_count):
            print(f'Warning: column {col}, {col_type} -> {df[col].dtypes} lost na values. Before: {na_count}, after: {new_na_count}')
        new_n_uniq = df[col].nunique(dropna=False)
        if (n_uniq != new_n_uniq):
            print(f'Warning: column {col}, {col_type} -> {df[col].dtypes} lost unique values. Before: {n_uniq}, after: {new_n_uniq}')

    end_mem = df.memory_usage(deep=deep).sum() / 1024 ** 2
    percent = 100 * (start_mem - end_mem) / start_mem
    print('Mem. usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(start_mem, end_mem, percent))
    return df


def chunk_predictor(X,model,n=100,verbose=True):
    predict_fold = KFold(n)
    result = np.zeros(X.shape[0])
    if verbose:
        for trn_idx, val_idx in tqdm_notebook(predict_fold.split(X)):
               result[val_idx] = model.predict(X.iloc[val_idx,:])
    else:
        for trn_idx, val_idx in predict_fold.split(X):
            result[val_idx] = model.predict(X.iloc[val_idx,:])
    return(result)

def truncate(number, digits):
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

def debugger_train(train,target):
    weather_col = int(np.where(train.columns == 'air_temperature')[0])
    mask = (train.iloc[:,weather_col:train.shape[1]].isnull().mean(axis=1)==1)

    if train.shape[0]!=len(target):
        raise ValueError('Shape from train and target are different')
    if any(mask):
        raise ValueError('Missing row in weather')
    return(train.columns)

def debugger_test(test,col):
    if train.shape[1]!=test.shape[1]:
        raise ValueError('Shape from train and target are different')
    if np.any(test.columns!=col):
        raise ValueError('Test column are different with train')

In [3]:
def clean_timestamps(df):
    """
    :param df: Dataframe containing a "timestamp" field which will be broken down in hour, year, day,...
    """
    df['timestamp'] = pd.to_datetime(df['timestamp'],format='%Y-%m-%d %H:%M:%S')
#     df["month"] = df["timestamp"].dt.month.astype(np.uint8)
    df["dayofweek"] = df["timestamp"].dt.dayofweek.astype(np.uint8)
#     df["dayofyear"] = df["timestamp"].dt.dayofyear.astype(np.uint8)
    df["day"] = df["timestamp"].dt.day.astype(np.uint8)
    df["hour"] = df["timestamp"].dt.hour.astype(np.uint8)
    df["weekend"] = df["timestamp"].dt.weekday.astype(np.uint8)
def free_space(del_list):
    for name in del_list:
        if not name.startswith('_'):
            del globals()[name]
    gc.collect()
def vector_apply(vec,fun):
    result = vec.copy()
    for i in tqdm_notebook(range(len(vec))):
        result[i] = fun(vec[i])
    return(result)

def res_fun(x):
    return(truncate(math.exp(x)-1,4))

In [4]:
def aligner_holiday(weather):
    timediff = {0:4,1:0,2:7,3:4,4:7,5:0,6:4,7:4,8:4,9:5,10:7,11:4,12:0,13:5,14:4,15:4}
    weather['time_diff'] = weather['site_id'].map(timediff)
    weather['time_diff'] = pd.to_timedelta(weather['time_diff'],'h')
    weather['timestamp'] = weather['timestamp'] - weather['time_diff']
    weather = city_mapper(weather)
    weather = holiday_mapper(weather)
    
    country_dic = {'UnitedStates':0,'England':1,'Canada':2,'Ireland':3} 
    holiday_dic = {False:0,True:1}
    
    weather['country'] = weather['country'].map(country_dic)
    weather['Holiday'] = weather['Holiday'].map(holiday_dic)
    
    return(weather.drop(['city','time_diff'],axis=1))

def city_mapper(weather):    
    country = {0:'UnitedStates',1:'England',2:'UnitedStates',3:'UnitedStates',4:'UnitedStates',
           5:'England',6:'UnitedStates',7:'Canada',8:'UnitedStates',9:'UnitedStates',10:'UnitedStates',
           11:'Canada',12:'Ireland',13:'UnitedStates',14:'UnitedStates',15:'UnitedStates'}
    city = {0:'Jacksonville',1:'London',2:'Phoenix',3:'Philadelphia',4:'San Francisco',
           5:'Loughborough',6:'Philadelphia',7:'Montreal',8:'Jacksonville',9:'San Antonio',10:'Las Vegas',
           11:'Montreal',12:'Dublin',13:'Minneapolis',14:'Philadelphia',15:'Pittsburgh'} 
    weather['country'] = weather['site_id'].map(country)
    weather['city'] = weather['site_id'].map(city)
    return(weather)

def holiday_mapper(weather):
    state = {'Jacksonville':'FL','London':None,'Phoenix':'AZ','Philadelphia':'PA','San Francisco':'CA',
     'Loughborough':None,'Philadelphia':'PA','Montreal':'QC','Jacksonville':'FL',
     'San Antonio':'TX','Las Vegas':'NV','Montreal':'QC','Dublin':None,'Minneapolis':'MN','Philadelphia':'PA',
     'Pittsburgh':'PA'}

    city_country_dic = {'Jacksonville':'UnitedStates','London':'England','Phoenix':'UnitedStates','Philadelphia':'UnitedStates',
                        'San Francisco':'UnitedStates','Loughborough':'England','Philadelphia':'UnitedStates',
                        'Montreal':'Canada','Jacksonville':'UnitedStates','San Antonio':'UnitedStates',
                        'Las Vegas':'UnitedStates','Montreal':'Canada',
                        'Dublin':'Ireland','Minneapolis':'UnitedStates','Philadelphia':'UnitedStates','Pittsburgh':'UnitedStates'}

    weather['Holiday'] = True
    for (city,abbr) in state.items():

        if city_country_dic[city] == 'UnitedStates':
            holiday = holidays.CountryHoliday(city_country_dic[city],state=abbr)
        else:
            holiday = holidays.CountryHoliday(city_country_dic[city],prov=abbr)

        weather.loc[weather.city==city,'Holiday'] = [x in holiday for x in weather.loc[weather.city==city,'timestamp']]
    return(weather)

def loader(mode='train'):
    
    if mode=='train':
        train = pd.read_csv(f'{path}train.csv')
        clean_timestamps(train)
        
        weather_train = pd.read_csv(f'{path_new}new_weather_train.csv')
        weather_add = pd.read_csv(f'{path_new}new_weather_test.csv')
        
        weather_train = weather_train.append(weather_add,ignore_index =True)        
        building_metadata = pd.read_csv(f'{path_new}new_building_metadata.csv')

        weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'],format='%Y-%m-%d %H:%M:%S')
        weather_train = aligner_holiday(weather_train)
        
        
        target = np.log1p(train.pop('meter_reading'))

        train = reduce_mem_usage_sd(train, obj_to_cat=False)
        weather_train = reduce_mem_usage_sd(weather_train, obj_to_cat=False)
        building_metadata = reduce_mem_usage_sd(building_metadata, obj_to_cat=False)
        
        print('Merging building_data...\n')
        train = train.merge(building_metadata, left_on = "building_id", right_on = "building_id", how = "left")

        print('Merging weather_data...\n')
        train = train.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
        gc.collect()
                        
        category_column = ['building_id','meter','day','hour','weekend','dayofweek','month','dayofyear','country','Holiday',
                           'site_id','primary_use','new_century_built','cloud_coverage',
                  'has_cloud','has_rain','has_sea','precipit_long','wind_compass','beaufort_scale','cloud_coverage_lag_24','wind_compass_lag_24',
                           'beaufort_scale_lag_24','cloud_coverage_lag_48','wind_compass_lag_48','beaufort_scale_lag_48']
        category_column = [x for x in category_column if x in train.columns]
        
        for col in tqdm_notebook(category_column):
            train[col] = train[col].astype('category')

        train = reduce_mem_usage_sd(train, obj_to_cat=False)
        
        mask = (train.site_id==0) & (train.meter==0) & (train.timestamp <='2016-05-20') 
        target = target.loc[~mask]
        train = train.loc[~mask]

        train.drop('timestamp',axis=1,inplace=True)
        
        train.reset_index(drop=True,inplace=True)
        target.reset_index(drop=True,inplace=True)

        return(train,target,category_column)
    
    if mode=='test':
        test = pd.read_csv(f'{path}test.csv')
        clean_timestamps(test)

        weather_test = pd.read_csv(f'{path_new}new_weather_test.csv')
        building_metadata = pd.read_csv(f'{path_new}new_building_metadata.csv')

        weather_test['timestamp'] = pd.to_datetime(weather_test['timestamp'],format='%Y-%m-%d %H:%M:%S')
        weather_test = aligner_holiday(weather_test)

        test = reduce_mem_usage_sd(test, obj_to_cat=False)
        weather_test = reduce_mem_usage_sd(weather_test, obj_to_cat=False)
        building_metadata = reduce_mem_usage_sd(building_metadata, obj_to_cat=False)
        
        print('Merging building_data...\n')
        test = test.merge(building_metadata, left_on = "building_id", right_on = "building_id", how = "left")

        print('Merging weather_data...\n')
        test = test.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
        gc.collect()

        category_column = ['building_id','meter','day','hour','weekend','dayofweek','month','dayofyear','country','Holiday',
                           'site_id','primary_use','new_century_built','cloud_coverage',
                  'has_cloud','has_rain','has_sea','precipit_long','wind_compass','beaufort_scale',
                           'cloud_coverage_lag_24','wind_compass_lag_24',
                           'beaufort_scale_lag_24','cloud_coverage_lag_48','wind_compass_lag_48','beaufort_scale_lag_48']
        category_column = [x for x in category_column if x in test.columns]

        for col in tqdm_notebook(category_column):
            test[col] = test[col].astype('category')

        test = reduce_mem_usage_sd(test, obj_to_cat=False)
        test.drop(['row_id','timestamp'],axis=1,inplace=True)
        return(test)


In [5]:
from itertools import groupby
from operator import itemgetter

def get_info_consecutive(data,max_len=24):
    return([x for x in [(next(g), sum(1 for i in g)+1) for k, g in groupby(enumerate(data), key=itemgetter(1))] if x[1]>max_len])

def get_position_consecutive(infol):
    pos = []
    for x in infol:
        pos += np.arange(start=x[0][0]+1,stop=x[0][0]+x[1]).tolist()
    return(pos)
    

In [6]:
train, target,category_column = loader()
col = debugger_train(train,target)
gc.collect()


Mem. usage decreased from 539.83 Mb to 289.19 Mb (46.4% reduction)



Mem. usage decreased from 57.80 Mb to 30.50 Mb (47.2% reduction)



Mem. usage decreased from  0.09 Mb to  0.03 Mb (68.7% reduction)
Merging building_data...

Merging weather_data...




Mem. usage decreased from 2043.69 Mb to 2043.69 Mb (0.0% reduction)


3

In [7]:
SEED = 42
seed_everything(SEED)
gc.collect()

0

In [8]:
%%time
strat =train['building_id'].astype(str) + train['meter'].astype(str)
le = LabelEncoder()
strat = le.fit_transform(strat).astype(np.int16)
free_space(['le'])
seed = 1
gc.collect()

CPU times: user 58.4 s, sys: 9.68 s, total: 1min 8s
Wall time: 1min 5s


0

In [9]:
def LGB_bayesian(
    num_leaves,  # int
    min_data_in_leaf,  # int
#    learning_rate,
    min_sum_hessian_in_leaf,    # int  
    colsample_bytree,
    subsample,
    lambda_l1,
    lambda_l2,
    min_gain_to_split,
    max_depth):
    
    gc.collect()
    # LightGBM expects next three parameters need to be integer. So we make them integer
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)
    
    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int

    param = {
        'num_leaves': num_leaves,
        'max_bin': 255,
        'n_jobs':-1,
        'max_depth':max_depth,
        'min_data_in_leaf': min_data_in_leaf,
        'learning_rate': .1,#learning_rate,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_gain_to_split': min_gain_to_split,
        'seed': seed,
        'objective': 'regression',
        'boosting_type':'gbdt',
        'metric':'rmse',
        'early_stopping_rounds':5, 
    }    
    score = 0
    n_fold = 2
    folds = StratifiedKFold(n_splits=n_fold, shuffle=False,random_state=0)
    gc.collect()
    for trn_idx, val_idx in folds.split(train,strat):
            gc.collect()
            train_x, train_y = train.iloc[trn_idx,:], target.iloc[trn_idx]
            valid_x, valid_y = train.iloc[val_idx,:], target.iloc[val_idx]
            model = lgb.train(param,lgb.Dataset(train_x, label=train_y,categorical_feature=category_column),
                              2000, valid_sets = lgb.Dataset(valid_x, label=valid_y,categorical_feature=category_column),verbose_eval=0) #
            score += math.sqrt(mean_squared_error(valid_y,chunk_predictor(valid_x,model,verbose=False)))/n_fold
    return(-score)

bounds_LGB = {
    'num_leaves': (32, 512), 
    'min_data_in_leaf': (20, 200),  
#    'learning_rate': (0.1, 0.3),
    'min_sum_hessian_in_leaf': (1e-5, 1e-2),    
    'colsample_bytree': (.4, 1),
    'subsample': (.4,1),
    'lambda_l1': (0, 2.0), 
    'lambda_l2': (0, 2.0), 
    'min_gain_to_split': (0, 1.0),
    'max_depth':(20,80),
}

lbg_bayes = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=1)
init_points = 5
n_iter = 22
print('-' * 133)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    lbg_bayes.maximize(init_points=init_points, n_iter=n_iter)


-------------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | colsam... | lambda_l1 | lambda_l2 | max_depth | min_da... | min_ga... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        | -1.121    |  0.6502   |  1.441    |  0.000228 |  38.14    |  46.42    |  0.09234  |  0.001871 |  197.9    |  0.6381   |
|  2        | -1.12     |  0.7233   |  0.8384   |  1.37     |  32.27    |  178.1    |  0.02739  |  0.006708 |  232.3    |  0.7352   |
|  3        | -1.12     |  0.4842   |  0.3962   |  1.601    |  78.1     |  76.42    |  0.6923   |  0.008765 |  461.4    |  0.451    |
|  4        | -1.122    |  0.4234   |  0.3397   |  1.756    |  25.9     |  95.8     |  0.9579   |  0.005336 |  364.1    |  0.5893   |
|  5        | -1.118    |  0.8119   |  1.669    |  0.03658  | 

In [10]:
lbg_bayes.max['target']


-1.1162845073908225

In [11]:
lbg_bayes.max['params']


{'colsample_bytree': 0.7699242354175626,
 'lambda_l1': 1.7233942541747633,
 'lambda_l2': 1.835065251306714,
 'max_depth': 69.45971425060898,
 'min_data_in_leaf': 30.792304068114845,
 'min_gain_to_split': 0.3262201710999464,
 'min_sum_hessian_in_leaf': 0.007097047333357153,
 'num_leaves': 510.37557713422353,
 'subsample': 0.7385647527529277}